In [1]:
import intake
from cmip6_preprocessing.preprocessing import combined_preprocessing
from cmip6_preprocessing.postprocessing import combine_datasets, merge_variables, match_metrics

In [2]:
variables = ['vo', 'thetao', 'so']
z_kwargs = {'consolidated': True, 'use_cftime': True}
kwargs = dict(zarr_kwargs=z_kwargs, storage_options={'token': 'anon'}, preprocess=combined_preprocessing, aggregate=False)
query = dict(experiment_id=['historical'], 
             table_id=['Omon'], 
             variable_id=variables,
             grid_label=['gr']
            )

col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")
cat = col.search(**query)
dd = cat.to_dataset_dict(**kwargs)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


In [3]:
def pick_first_member(ds_list, **kwargs):
    return ds_list[0]

ddnow = merge_variables(dd)
ddnow = combine_datasets(
    ddnow,
    pick_first_member,
    match_attrs=['source_id', 'grid_label', 'experiment_id', 'table_id']
)
list(ddnow.keys())

/srv/conda/envs/notebook/lib/python3.9/site-packages/cmip6_preprocessing/postprocessing.py:122: UserWarning: CMIP.NCC.NorCPM1.historical.r19i1p1f1.Omon.gr.none failed to combine with :indexes along dimension 'time' are not equal
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/srv/conda/envs/notebook/lib/python3.9/site-packages/cmip6_preprocessing/postprocessing.py:122: UserWarning: CMIP.NCC.NorCPM1.historical.r22i1p1f1.Omon.gr.none failed to combine with :indexes along dimension 'time' are not equal
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/srv/conda/envs/notebook/lib/python3.9/site-packages/cmip6_preprocessing/postprocessing.py:122: UserWarning: CMIP.NCC.NorCPM1.historical.r5i1p1f1.Omon.gr.none failed to combine with :indexes along dimension 'time' are not equal
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/srv/conda/envs/notebook/lib/python3.9/site-packages/cmip6_preprocessing/postprocessing.py:122: UserW

['GISS-E2-1-H.gr.historical.Omon',
 'NorESM2-LM.gr.historical.Omon',
 'CESM2.gr.historical.Omon',
 'NorESM2-MM.gr.historical.Omon',
 'MRI-ESM2-0.gr.historical.Omon',
 'GFDL-ESM4.gr.historical.Omon',
 'E3SM-1-0.gr.historical.Omon',
 'CESM2-WACCM.gr.historical.Omon',
 'GISS-E2-2-H.gr.historical.Omon',
 'GFDL-CM4.gr.historical.Omon',
 'EC-Earth3-Veg.gr.historical.Omon',
 'E3SM-1-1-ECA.gr.historical.Omon',
 'E3SM-1-1.gr.historical.Omon',
 'CESM2-FV2.gr.historical.Omon',
 'CESM2-WACCM-FV2.gr.historical.Omon']

In [5]:
ddsub = {}
for d in ddnow.keys():
    vs = ddnow[d].data_vars
    if ('vo' in vs) & ('thetao' in vs) & ('so' in vs):
        ddsub[d] = ddnow[d]
list(ddsub.keys())

['GISS-E2-1-H.gr.historical.Omon',
 'E3SM-1-0.gr.historical.Omon',
 'E3SM-1-1-ECA.gr.historical.Omon',
 'E3SM-1-1.gr.historical.Omon']